In [ ]:
!pip install pytesseract evaluate tqdm transformers datasets rouge-score accelerate nltk tensorboard jupyter-black py7zr --upgrade
!apt-get install git --yes
!apt-get install git-lfs --yes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:

In [ ]:
import json
import pandas
import jupyter_black
from datetime import timedelta
from datasets import Dataset

In [ ]:
with open("data.json", "r") as f:
    data = json.load(f)

# create a dataframe
df = pandas.DataFrame(data)
df

,id,task,question,maximum_score,scoring_guide,answer,score
0,1,Rate the answer to the question.,Explain the difference between malloc() and ne...,6,"[{'point': 2, 'criteria': 'malloc() is a C fun...",malloc() is a C function used for dynamic memo...,6
1,2,Rate the answer to the question.,Explain the difference between malloc() and ne...,6,"[{'point': 2, 'criteria': 'malloc() is a C fun...",malloc() is a C function used for dynamic memo...,4
2,3,Rate the answer to the question.,Explain the difference between malloc() and ne...,6,"[{'point': 2, 'criteria': 'malloc() is a C fun...","After calling malloc(), free() must be also ca...",2
3,4,Rate the answer to the question.,Explain the difference between malloc() and ne...,6,"[{'point': 2, 'criteria': 'malloc() is a C fun...",The memory allocated with new must be dealloca...,0
4,5,Rate the answer to the question.,What is the purpose of the 'const' keyword in ...,3,"[{'point': 2, 'criteria': 'The 'const' keyword...",The 'const' keyword in C++ is used to indicate...,2
5,6,Rate the answer to the question.,What is the purpose of the 'const' keyword in ...,3,"[{'point': 2, 'criteria': 'The 'const' keyword...",The 'const' keyword in C++ is used to indicate...,2


In [ ]:
data = Dataset.from_pandas(df).train_test_split(test_size=0.1)
print(data)

DatasetDict({
    train: Dataset({
        features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score'],
        num_rows: 5
    })
    test: Dataset({
        features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score'],
        num_rows: 1
    })
})


Ezidáig az adatok lettek egy datasetbe feldolgozva, innentől a modell felkészítése jön a trainingre

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
MODEL_NAME = "C++ teacher AI"  # the name of your model
MODEL_ID = "google/flan-t5-base"  # the id of the base model we will train (can be small, base, large, xl, etc.) (the bigger - the more GPU memory you need)
NUM_TRAIN_EPOCHS = 4  # number of epochs to train

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)